In [75]:
# Importing the required libraries
import os
import cv2
import glob
import zipfile
import numpy as np
from tqdm import tqdm
from urllib import request
from google.colab import drive

In [76]:
#you need to replace this by the location of the dataset on your drive
# datasetloc="/content/drive/MyDrive/Hemangini Projects/dataset"
#! wget -q https://storage.googleapis.com/drive-bulk-export-anonymous/20220428T122708Z/4133399871716478688/aa76fc5c-9354-4b5f-ae56-aca2048e1129/1/acf3a9e4-29f2-415b-86b6-f7ba375e19ee?authuser -O dataset.zip
# drive.mount('/content/drive')
# datasetloc="/content/drive/MyDrive/Colab Notebooks/dataset"

# ! wget -q https://tinyurl.com/r5ut6tjx -O dataset.zip
# ! unzip -oqq dataset
# datasetloc="./dataset"


#if not os.path.exists("./dataset.zip"):
#    response = request.urlretrieve("https://drive.google.com/file/d/1z_B2zhcjl0heRbdRGUTQKwC82WqDjBLb/view?usp=sharing", "dataset.zip")
#    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
#        zip_ref.extractall(".")
#datasetloc="./dataset"

#-----------------------------------------------------------------------------------------------
# The dataset has been obtained from Kaggle with the dataset called Real and Fake Face Detection
# It has been published by the Computational Intelligence and Photography Lab
# Department of Computer Science, Yonsei University
# https://www.kaggle.com/datasets/ciplab/real-and-fake-face-detection
drive.mount('/content/drive')
datasetloc = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [77]:
# Return a list of path for the images in the folder as an array
fakefiles=glob.glob(datasetloc+"/training_fake/*.jpg")

In [78]:
#reading the FAKE images and labelling them as 1
#tqdm helps to keep track of progress
data=[]
y=[]
for f in tqdm(fakefiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  
  data.append(img)
  y.append(1)

100%|██████████| 970/970 [00:15<00:00, 60.88it/s]


In [79]:
realfiles=glob.glob(datasetloc+"/training_real/*.jpg")

In [80]:
# Reading the REAL images and labelling them as 0
# Resize image so we can stay consistent with the uniformity throughout
for f in tqdm(realfiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  data.append(img)
  y.append(0)
  

100%|██████████| 1081/1081 [00:18<00:00, 56.99it/s]


In [81]:
#convering lists into arrays // Arrays of all pixelated arrays
data = np.asarray(data)
y = np.asarray(y)

In [82]:
# We do a random split on the dataset at 80% and 20% with; 80% of the data would be training the model and remaining 20% of the data would be checking the model performance
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [83]:
#importing the libraries required for our cnn model
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

In [84]:
# Sketched Model Architecture for FAKE an REAL image detection
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (128,128,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])
# In the last layer if the value is Greater then 0.5 it is fake image and if the value is less then 0.5 it is real image

In [85]:
#fitting our model on the dataset, train the model using X train and Y train
model.fit(X_train,y_train,epochs=10,batch_size=40)

Epoch 1/10
41/41 [==============================] - 53s 1s/step - loss: 50.4948 - accuracy: 0.5159
Epoch 2/10
41/41 [==============================] - 51s 1s/step - loss: 0.6974 - accuracy: 0.5104
Epoch 3/10
41/41 [==============================] - 51s 1s/step - loss: 0.6947 - accuracy: 0.5006
Epoch 4/10
41/41 [==============================] - 53s 1s/step - loss: 0.6969 - accuracy: 0.5207
Epoch 5/10
41/41 [==============================] - 53s 1s/step - loss: 0.6940 - accuracy: 0.5256
Epoch 6/10
41/41 [==============================] - 52s 1s/step - loss: 0.6945 - accuracy: 0.5091
Epoch 7/10
41/41 [==============================] - 52s 1s/step - loss: 0.6909 - accuracy: 0.5244
Epoch 8/10
41/41 [==============================] - 52s 1s/step - loss: 0.6881 - accuracy: 0.5311
Epoch 9/10
41/41 [==============================] - 52s 1s/step - loss: 0.6850 - accuracy: 0.5323
Epoch 10/10
41/41 [==============================] - 52s 1s/step - loss: 0.6853 - accuracy: 0.5415


In [86]:
ypredtrain=model.predict(X_train)
ypredtest=model.predict(X_test)

In [87]:
# Using the train model we made predictions
yptrain=[]
yptest=[]
for y in ypredtrain:
  if(y>=0.5):
    yptrain.append(1)
  else:
    yptrain.append(0)
for y in ypredtest:
  if(y>=0.5):
    yptest.append(1)
  else:
    yptest.append(0)


In [88]:
from sklearn.metrics import accuracy_score
print("the accuracy score for training set is",accuracy_score(y_train, yptrain))
print("the accuracy score for test set is",accuracy_score(y_test, yptest))

the accuracy score for training set is 0.5573170731707318
the accuracy score for test set is 0.5328467153284672
